## This notebook is part of Hadoop and Spark training delivered by CERN IT
### Spark Structured Streaming Hands-On Lab
Contact: Luca.Canali@cern.ch

Run this notebook from Jupyter with Python kernel
- When using on CERN SWAN, do not attach the notebook to a Spark cluster, but rather run locally on the SWAN container
- If running this outside CERN SWAN, please make sure to have PySpark installed: `pip install pyspark`

In [ ]:
# Create Spark Session, you need this to work with Spark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("my streaming test app")  \
        .master("local[*]") \
        .config("spark.driver.memory","2g") \
        .config("spark.ui.showConsoleProgress", "false") \
        .getOrCreate()

In [ ]:
# sets the path to the directory with datafiles
PATH = "../data/streaming/"

schema = "timestamp int, name string, value double"

In [ ]:
# Define the input part of the streaming pipeline
# This reads all the .csv files in a given directory
# It checks continuosly for arrival of new files

input_path = PATH + "*.csv"
input_stream = (spark.readStream.format("csv")
               .option("header","true")
               .schema(schema)
               .option("path", input_path)
               .load())

In [ ]:
# Defines an output stream of the pipeline, this writes data to a view in memory
# Use for testing, in a real case you would write to files and/or Kafka
#
# Delete the checkpoint dir if it already exists
# ! rm -r myStreamingCheckPoint1

raw_stream = (input_stream.writeStream 
             .queryName("data_read")
             .outputMode("append")
             .format("memory")
             .option("checkpointLocation", "myStreamingCheckPoint1") 
             .start())

In [ ]:
spark.table("data_read").printSchema()

In [ ]:
# Query the output table
# Run this multiple times, as you add csv files with data in the input_path directory

spark.sql("select * from data_read").show()

In [ ]:
raw_stream.status

In [ ]:
# This maps the input_stream to a temporary view, so that we can work with it using SQL
input_stream.createOrReplaceTempView("input_stream")

In [ ]:
# Use Spark SQL to describe the aggregation and tranformation on streaming data
df = spark.sql("""
select name||'_aggregated' as name_aggregated, count(*) as n_points, sum(value) sum_values 
from input_stream 
group by name""")

In [ ]:
# Defines another output stream for the pipeline
aggregated_stream = (df.writeStream
                    .queryName("data_aggregated")
                    .outputMode("complete")
                    .format("memory")
                    .option("checkpointLocation", "myStreamingCheckPoint2") 
                    .start())

In [ ]:
# Query the table with aggregated data, this is updated as new data arrives in the input pipeline
spark.sql("select * from data_aggregated").show()

----

## Simulate the arrival of streaming data
Add more data in form of .csv files to the input_path folder
and run the queries of the output streams again

In [ ]:
# Query the output table
# Run this multiple times, as you add csv files with data in the input_path directory

spark.sql("select * from data_read").show()

In [ ]:
# Query the table with aggregated data
# this is updated as new data arrives in the input pipeline

spark.sql("select * from data_aggregated").show()

In [ ]:
# stop everything
raw_stream.stop()
aggregated_stream.stop()
spark.stop()